In [2]:
import cv2
import numpy as np
import os
import json
import ast

In [3]:
# Make a list of all folders in a folder
def get_folder_list(folder):
    return [name for name in os.listdir(folder) if os.path.isdir(os.path.join(folder, name))]

In [4]:
def get_BB(folder, set_list):
    bbox_dict = {}
    for s in set_list:
        seq = cv2.VideoCapture(f"{folder}/{s}/Image#%04d.jpg")
        if not seq.isOpened():
            print('error reading image sequence')
            break
        valid, frame = seq.read()
        if not valid:
            print('error reading image')
            break
        bbox = cv2.selectROIs("Select Rois",frame)
        bbox = [tuple(x) for x in bbox]
        bbox_dict[s] = bbox

    cv2.destroyAllWindows()
    return bbox_dict

In [5]:
def track(folder, tracker, bbox, output_file):
    '''
    Track an object initially denoted by a bounding box (bbox)
    and write out the location in an output file
    '''
    seq = cv2.VideoCapture(f"{folder}/Image#%04d.jpg")
    if not seq.isOpened():
        print('error reading image sequence')
    valid, frame = seq.read()
    if not valid:
        print('error reading image')

    # Initialize tracker with first frame and bounding box
    valid = tracker.init(frame, bbox)

    data = []
    while True:
        # Read a new frame
        valid, frame = seq.read()
        if not valid:
            break

        # Start timer
        timer = cv2.getTickCount()

        # Update tracker
        valid, bbox = tracker.update(frame)

        # Calculate Frames per second (FPS)
        fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer)

        # Draw bounding box
        if valid:
            # Tracking success
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame, p1, p2, (255, 0, 0), 2, 1)
        else:
            # Tracking failure
            cv2.putText(frame, "Tracking failure detected", (100, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
        x = [timer]
        x.extend(bbox)
        # data
        data.append(np.array(x))
        # Display result
        cv2.imshow("Tracking", frame)

        # Exit if ESC pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):  # if press SPACE bar
            break
    
    # Save data
    np.savetxt(output_file, data, delimiter=',')
    cv2.destroyAllWindows()

In [6]:
def dict_to_json(dict, filename):
    '''Saves a dictionary to a json file'''
    with open(filename, 'w') as f:
        json.dump(dict, f)

def json_to_dict(filename):
    '''Loads a json file into a dictionary'''
    with open(filename) as f:
        return json.load(f)


In [7]:
def dict_to_txt(dict, filename):
    '''Saves a dictionary to a txt file'''
    with open(filename, 'w') as f:
        for key in dict:
            f.write(f"{key}:{dict[key]}\n")

def txt_to_dict(filename):
    '''Converts a txt file into a dictionary'''
    with open(filename) as f:
        return {line.split(':')[0]: ast.literal_eval(line.split(':')[1][:-1]) for line in f}

# Track the Blobs

In [12]:
# Pre-processing
set_list = get_folder_list('data')
bbox_dict = get_BB('data', set_list)
dict_to_txt(bbox_dict, 'bbox_dict.txt')

In [13]:
# Tracking
bbox_dict = txt_to_dict('bbox_dict.txt')
tracker = cv2.TrackerKCF_create()

for s in set_list:
    i = 0
    for bbox in bbox_dict[s]:
        print(s, i, bbox)
        tracker = cv2.TrackerKCF_create()
        output = track(f"data/{s}", tracker, bbox, f"output/{s}-{i}.txt")
        i += 1

s1 0 (661, 354, 111, 122)
s10 0 (533, 456, 111, 102)
s2 0 (894, 405, 161, 167)
s3 0 (728, 577, 108, 70)
s4 0 (408, 659, 87, 79)
s5 0 (509, 419, 70, 71)
s7 0 (814, 279, 129, 112)
s8 0 (354, 411, 204, 155)
s9 0 (305, 214, 60, 67)
Set10a 0 (434, 172, 248, 234)
Set10a 1 (315, 442, 72, 77)
Set10a 2 (0, 522, 53, 67)
Set10a 3 (815, 293, 89, 92)
Set10a 4 (957, 79, 66, 64)
Set10a 5 (833, 616, 85, 99)
Set1a 0 (245, 97, 196, 148)
Set1a 1 (761, 217, 47, 43)
Set1a 2 (1062, 166, 82, 75)
Set1a 3 (894, 375, 77, 78)
Set1a 4 (902, 539, 97, 87)
set1b 0 (380, 207, 128, 104)
set1b 1 (679, 350, 79, 73)
set1b 2 (660, 487, 66, 88)
set1b 3 (443, 551, 183, 140)
set1b 4 (57, 539, 81, 75)
set1b 5 (1145, 399, 106, 94)
Set2a 0 (471, 362, 69, 65)
Set2a 1 (625, 183, 104, 91)
Set2a 2 (101, 530, 77, 70)
Set2a 3 (867, 622, 151, 134)
set2b 0 (647, 327, 84, 77)
set2b 1 (1089, 215, 76, 68)
set2b 2 (1023, 507, 53, 60)
Set3a 0 (590, 217, 54, 64)
Set3a 1 (457, 523, 50, 59)
Set3a 2 (877, 633, 148, 137)
set3b 0 (382, 298, 62, 5